# Load ngữ liệu và tách từ

In [26]:
import pandas as pd
import os
from pyvi import ViTokenizer

# --- CẤU HÌNH ---
INPUT_DIR = "./UIT-ViHSD" # Folder gốc
OUTPUT_DIR = "./UIT-ViHSD-preprocessed" # Folder đích

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# --- LOAD DỮ LIỆU GỐC ---
print("⏳ Đang đọc dữ liệu gốc...")
df_train = pd.read_csv(os.path.join(INPUT_DIR, "train.csv"))
df_dev = pd.read_csv(os.path.join(INPUT_DIR, "dev.csv"))
df_test = pd.read_csv(os.path.join(INPUT_DIR, "test.csv"))

# Xử lý NaN ban đầu (bắt buộc để code không lỗi)
for df in [df_train, df_dev, df_test]:
    df.dropna(subset=['free_text'], inplace=True)
    df['free_text'] = df['free_text'].astype(str)

# --- TÁCH TỪ (SEGMENTATION) ---
print("⏳ Đang thực hiện tách từ (PyVi)...")
def safe_segment(text):
    try:
        return ViTokenizer.tokenize(text)
    except:
        return text

# Áp dụng
df_train['free_text'] = df_train['free_text'].apply(safe_segment)
df_dev['free_text'] = df_dev['free_text'].apply(safe_segment)
df_test['free_text'] = df_test['free_text'].apply(safe_segment)

print("✅ Đã tách từ xong.")

⏳ Đang đọc dữ liệu gốc...
⏳ Đang thực hiện tách từ (PyVi)...
✅ Đã tách từ xong.


# Loại bỏ Stopword với cơ chế "Safety Check"

In [27]:
# --- CẤU HÌNH STOPWORDS ---
STOPWORD_FILE = "vietnamese-stopwords.txt" 

# Load stopwords
stopword_set = set()
try:
    with open(STOPWORD_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            word = line.strip()
            if word:
                # Chuyển stopword sang dạng segmented để khớp dữ liệu (ví dụ: "bởi vì" -> "bởi_vì")
                stopword_set.add(word.replace(' ', '_'))
    print(f"✅ Đã load {len(stopword_set)} stopwords.")
except FileNotFoundError:
    print("⚠️ Không tìm thấy file stopwords. Bỏ qua bước này.")

# --- HÀM LOẠI BỎ AN TOÀN ---
def remove_stopword_safe(text, stop_set):
    if not text: return ""
    
    tokens = text.split()
    # Giữ lại các từ KHÔNG nằm trong danh sách stopword
    clean_tokens = [t for t in tokens if t.lower() not in stop_set]
    
    # KỸ THUẬT AN TOÀN:
    # Nếu xóa xong mà rỗng (tức là câu toàn stopword) -> Trả về câu gốc (tokens ban đầu)
    # Nếu còn từ -> Trả về câu đã clean
    if len(clean_tokens) == 0:
        return text # <--- GIỮ LẠI GỐC
    
    return " ".join(clean_tokens)

# --- THỰC THI ---
if len(stopword_set) > 0:
    print("⏳ Đang loại bỏ stopwords (chế độ an toàn)...")
    df_train['free_text'] = df_train['free_text'].apply(lambda x: remove_stopword_safe(x, stopword_set))
    df_dev['free_text'] = df_dev['free_text'].apply(lambda x: remove_stopword_safe(x, stopword_set))
    df_test['free_text'] = df_test['free_text'].apply(lambda x: remove_stopword_safe(x, stopword_set))
    print("✅ Hoàn tất.")

✅ Đã load 1942 stopwords.
⏳ Đang loại bỏ stopwords (chế độ an toàn)...
✅ Hoàn tất.


In [28]:
def check_empty_rows(df, name):
    # Đếm số dòng rỗng hoặc chỉ chứa khoảng trắng
    count = df[df['free_text'].str.strip() == ''].shape[0]
    print(f"📊 [{name}] Số dòng rỗng: {count}")
    return count

print("--- KIỂM TRA DỮ LIỆU RỖNG ---")
c1 = check_empty_rows(df_train, "Train")
c2 = check_empty_rows(df_dev, "Dev")
c3 = check_empty_rows(df_test, "Test")

if c1 == 0 and c2 == 0 and c3 == 0:
    print("\n🎉 TUYỆT VỜI! Không có dòng nào bị rỗng. Dữ liệu bảo toàn 100%.")
else:
    print("\n⚠️ Vẫn còn dòng rỗng (Lỗi bất thường).")

# Lưu tạm kết quả bước này
df_train.to_csv(os.path.join(OUTPUT_DIR, "train_step1_stopword.csv"), index=False)
df_dev.to_csv(os.path.join(OUTPUT_DIR, "dev_step1_stopword.csv"), index=False)
df_test.to_csv(os.path.join(OUTPUT_DIR, "test_step1_stopword.csv"), index=False)

--- KIỂM TRA DỮ LIỆU RỖNG ---
📊 [Train] Số dòng rỗng: 0
📊 [Dev] Số dòng rỗng: 0
📊 [Test] Số dòng rỗng: 0

🎉 TUYỆT VỜI! Không có dòng nào bị rỗng. Dữ liệu bảo toàn 100%.


# Chuyển hết sang lower case

In [29]:
import pandas as pd
import os

# --- CẤU HÌNH ---
DIR = "./UIT-ViHSD-preprocessed"

# Tên file đầu vào (Kết quả của bước Stopwords vừa chạy)
INPUT_FILES = {
    'train': "train_step1_stopword.csv",
    'dev': "dev_step1_stopword.csv",
    'test': "test_step1_stopword.csv"
}

# Tên file đầu ra
OUTPUT_FILES = {
    'train': "train_step2_lower.csv",
    'dev': "dev_step2_lower.csv",
    'test': "test_step2_lower.csv"
}

def to_lowercase_and_save():
    print("⏳ Đang thực hiện chuyển đổi chữ thường (Lowercase)...")
    
    for split_name, file_name in INPUT_FILES.items():
        input_path = os.path.join(DIR, file_name)
        
        # Kiểm tra file tồn tại
        if not os.path.exists(input_path):
            print(f"❌ Lỗi: Không tìm thấy file {file_name}. Vui lòng chạy lại bước Stopwords trước.")
            continue
            
        # 1. Đọc dữ liệu
        df = pd.read_csv(input_path)
        
        # Xử lý NaN (chuyển thành chuỗi rỗng để không lỗi, dù bước trước đã xử lý)
        df['free_text'] = df['free_text'].fillna("").astype(str)
        
        # 2. Chuyển chữ thường
        # Đây là bước quan trọng nhất
        df['free_text'] = df['free_text'].str.lower()
        
        # 3. Kiểm tra lại rỗng (Sanity Check)
        empty_count = df[df['free_text'].str.strip() == ''].shape[0]
        print(f"   - [{split_name}] Đã chuyển đổi xong. Số dòng rỗng: {empty_count}")
        
        # 4. Lưu file
        output_path = os.path.join(DIR, OUTPUT_FILES[split_name])
        df.to_csv(output_path, index=False)
        print(f"     -> Đã lưu: {OUTPUT_FILES[split_name]}")

# --- THỰC THI ---
to_lowercase_and_save()

print("\n✅ Hoàn tất bước Lowercase. Bạn đã sẵn sàng cho bước tiếp theo (Xử lý Regex).")

⏳ Đang thực hiện chuyển đổi chữ thường (Lowercase)...
   - [train] Đã chuyển đổi xong. Số dòng rỗng: 0
     -> Đã lưu: train_step2_lower.csv
   - [dev] Đã chuyển đổi xong. Số dòng rỗng: 0
     -> Đã lưu: dev_step2_lower.csv
   - [test] Đã chuyển đổi xong. Số dòng rỗng: 0
     -> Đã lưu: test_step2_lower.csv

✅ Hoàn tất bước Lowercase. Bạn đã sẵn sàng cho bước tiếp theo (Xử lý Regex).


# Loại bỏ url

In [30]:
import pandas as pd
import re
import os

# --- CẤU HÌNH ---
DIR = "./UIT-ViHSD-preprocessed"

# Load dữ liệu từ bước trước (Lowercase)
print("⏳ Đang đọc dữ liệu từ bước Lowercase...")
df_train = pd.read_csv(os.path.join(DIR, "train_step2_lower.csv"))
df_dev = pd.read_csv(os.path.join(DIR, "dev_step2_lower.csv"))
df_test = pd.read_csv(os.path.join(DIR, "test_step2_lower.csv"))

# Đảm bảo không lỗi NaN
for df in [df_train, df_dev, df_test]:
    df['free_text'] = df['free_text'].fillna("").astype(str)

# --- XỬ LÝ URL ---
print("\n👉 BƯỚC 1: LOẠI BỎ URL (http://..., www...)")

def remove_url(text):
    # Regex bắt các đường link
    return re.sub(r'http\S+|www\.\S+', '', text).strip()

# Áp dụng
df_train['free_text'] = df_train['free_text'].apply(remove_url)
df_dev['free_text'] = df_dev['free_text'].apply(remove_url)
df_test['free_text'] = df_test['free_text'].apply(remove_url)

# --- KIỂM TRA RỖNG ---
def check_empty(df, name):
    count = df[df['free_text'].str.strip() == ''].shape[0]
    return count

print("📊 Kiểm tra dữ liệu rỗng sau khi bỏ URL:")
e1 = check_empty(df_train, "Train")
e2 = check_empty(df_dev, "Dev")
e3 = check_empty(df_test, "Test")

print(f"   - Train: {e1} dòng rỗng")
print(f"   - Dev:   {e2} dòng rỗng")
print(f"   - Test:  {e3} dòng rỗng")

⏳ Đang đọc dữ liệu từ bước Lowercase...

👉 BƯỚC 1: LOẠI BỎ URL (http://..., www...)
📊 Kiểm tra dữ liệu rỗng sau khi bỏ URL:
   - Train: 0 dòng rỗng
   - Dev:   0 dòng rỗng
   - Test:  0 dòng rỗng


# Loại bỏ hashtag

In [31]:
# --- XỬ LÝ HASHTAG ---
print("\n👉 BƯỚC 2: LOẠI BỎ KÝ TỰ HASHTAG (Xóa dấu #, giữ lại chữ)")

def remove_hashtag_symbol(text):
    # Chỉ thay thế dấu # bằng chuỗi rỗng
    return text.replace('#', '').strip()

# Áp dụng
df_train['free_text'] = df_train['free_text'].apply(remove_hashtag_symbol)
df_dev['free_text'] = df_dev['free_text'].apply(remove_hashtag_symbol)
df_test['free_text'] = df_test['free_text'].apply(remove_hashtag_symbol)

# --- KIỂM TRA RỖNG ---
print("📊 Kiểm tra dữ liệu rỗng sau khi bỏ Hashtag:")
e1 = check_empty(df_train, "Train")
e2 = check_empty(df_dev, "Dev")
e3 = check_empty(df_test, "Test")

print(f"   - Train: {e1} dòng rỗng")
print(f"   - Dev:   {e2} dòng rỗng")
print(f"   - Test:  {e3} dòng rỗng")


👉 BƯỚC 2: LOẠI BỎ KÝ TỰ HASHTAG (Xóa dấu #, giữ lại chữ)
📊 Kiểm tra dữ liệu rỗng sau khi bỏ Hashtag:
   - Train: 0 dòng rỗng
   - Dev:   0 dòng rỗng
   - Test:  0 dòng rỗng


# Loại bỏ mention

In [32]:
# --- XỬ LÝ MENTION ---
print("\n👉 BƯỚC 3: LOẠI BỎ MENTION TAG (@user)")

def remove_mention(text):
    # Regex bắt @ đi liền với các ký tự không phải khoảng trắng
    return re.sub(r'@\S+', '', text).strip()

# Áp dụng
df_train['free_text'] = df_train['free_text'].apply(remove_mention)
df_dev['free_text'] = df_dev['free_text'].apply(remove_mention)
df_test['free_text'] = df_test['free_text'].apply(remove_mention)

# --- KIỂM TRA RỖNG LẦN CUỐI ---
print("📊 Kiểm tra dữ liệu rỗng sau khi bỏ Mention (FINAL CHECK):")
e1_final = check_empty(df_train, "Train")
e2_final = check_empty(df_dev, "Dev")
e3_final = check_empty(df_test, "Test")

print(f"   - Train: {e1_final} dòng rỗng")
print(f"   - Dev:   {e2_final} dòng rỗng")
print(f"   - Test:  {e3_final} dòng rỗng")

# --- LƯU KẾT QUẢ CUỐI CÙNG ---
if e1_final == 0 and e3_final == 0:
    print("\n✅ Dữ liệu hoàn hảo! Đang lưu file final...")
else:
    print("\n⚠️ Có một số dòng bị rỗng (do câu chỉ chứa toàn link/mention). Đang lưu file final...")

# Định nghĩa đường dẫn file chuẩn
final_train = os.path.join(DIR, "train.csv")
final_dev = os.path.join(DIR, "dev.csv")
final_test = os.path.join(DIR, "test.csv")

# Lưu file
df_train.to_csv(final_train, index=False)
df_dev.to_csv(final_dev, index=False)
df_test.to_csv(final_test, index=False)

print(f"🎉 Đã hoàn tất toàn bộ quy trình Preprocessing!")
print(f"📂 File kết quả: {DIR}")


👉 BƯỚC 3: LOẠI BỎ MENTION TAG (@user)
📊 Kiểm tra dữ liệu rỗng sau khi bỏ Mention (FINAL CHECK):
   - Train: 0 dòng rỗng
   - Dev:   0 dòng rỗng
   - Test:  0 dòng rỗng

✅ Dữ liệu hoàn hảo! Đang lưu file final...
🎉 Đã hoàn tất toàn bộ quy trình Preprocessing!
📂 File kết quả: ./UIT-ViHSD-preprocessed
